In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [23]:
db = sqlite3.connect('data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv',
                       chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('hop_team', db, if_exists = 'append', index = False)
    
#db.execute('CREATE INDEX from_npi ON hop_team(from_npi)')

db.close()

0it [00:00, ?it/s]

In [26]:
db.execute('CREATE INDEX from_npi ON hop_team(from_npi)')

In [28]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM hop_team LIMIT 5"

hop_team_sqlite = pd.read_sql(query, db)

db.close()

In [29]:
hop_team_sqlite.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


In [31]:
db = sqlite3.connect('data/hop_team.sqlite')
cbsa = pd.read_excel('data/ZIP_CBSA_122021.xlsx')
cbsa.columns = [x.lower().replace(' ', '_') for x in cbsa.columns]

cbsa.to_sql('cbsa', db, if_exists = 'append', index = False)

#for chunk in tqdm(pd.read_excel('data/ZIP_CBSA_122021.xlsx'), chunksize = 10000):
 #   chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
#    chunk.to_sql('cbsa', db, if_exists = 'append', index = False)

#db.execute('CREATE INDEX from_npi ON hop_team(from_npi)')

db.close()

In [ ]:
db.execute('CREATE INDEX cbsa ON cbsa(cbsa)')

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,683,41900,SAN GERMAN,PR,0.999842,1.0,1.0,0.999855
1,683,32420,SAN GERMAN,PR,0.000158,0.0,0.0,0.000145
2,923,41980,SAN JUAN,PR,1.000000,1.0,1.0,1.000000
3,1010,44140,BRIMFIELD,MA,0.976896,1.0,1.0,0.977816
4,1010,49340,BRIMFIELD,MA,0.023104,0.0,0.0,0.022184


In [47]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM cbsa LIMIT 5"

cbsa_sqlite = pd.read_sql(query, db)

db.close()

cbsa_sqlite.head()

DatabaseError: Execution failed on sql 'SELECT * FROM cbsa LIMIT 5': database is locked

In [34]:
db.execute('CREATE INDEX cbsa ON cbsa(cbsa)')

OperationalError: there is already a table named cbsa

In [65]:
db = sqlite3.connect('data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv',
                  chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('npi', db, if_exists = 'append', index = False)

db.close()

0it [00:00, ?it/s]

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': database is locked

In [ ]:
db = sqlite3.connect('data/hop_team.sqlite')
taxonomy = pd.DataFrame()
for chunk in tqdm(pd.read_csv('data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv', chunksize = 10000)):
    taxonomy_switch = pd.melt(chunk, id_vars=['NPI'], value_vars = ['Healthcare Provider Primary Taxonomy Switch_1',
                                            'Healthcare Provider Primary Taxonomy Switch_2',
                                            'Healthcare Provider Primary Taxonomy Switch_3',
                                            'Healthcare Provider Primary Taxonomy Switch_4',
                                            'Healthcare Provider Primary Taxonomy Switch_5',
                                            'Healthcare Provider Primary Taxonomy Switch_6',
                                            'Healthcare Provider Primary Taxonomy Switch_7',
                                            'Healthcare Provider Primary Taxonomy Switch_8',
                                            'Healthcare Provider Primary Taxonomy Switch_9',
                                            'Healthcare Provider Primary Taxonomy Switch_10',
                                            'Healthcare Provider Primary Taxonomy Switch_11',
                                            'Healthcare Provider Primary Taxonomy Switch_12',
                                            'Healthcare Provider Primary Taxonomy Switch_13',
                                            'Healthcare Provider Primary Taxonomy Switch_14',
                                            'Healthcare Provider Primary Taxonomy Switch_15'])
    taxonomy_switch = taxonomy_switch.rename(columns={'value':'taxonomy_switch'})
    taxonomy_code=pd.melt(chunk, id_vars=['NPI'], value_vars = ['Healthcare Provider Taxonomy Code_1',
                                           'Healthcare Provider Taxonomy Code_2',
                                           'Healthcare Provider Taxonomy Code_3',
                                           'Healthcare Provider Taxonomy Code_4',
                                           'Healthcare Provider Taxonomy Code_5',
                                           'Healthcare Provider Taxonomy Code_6',
                                           'Healthcare Provider Taxonomy Code_7',
                                           'Healthcare Provider Taxonomy Code_8',
                                           'Healthcare Provider Taxonomy Code_9',
                                           'Healthcare Provider Taxonomy Code_10',
                                           'Healthcare Provider Taxonomy Code_11',
                                           'Healthcare Provider Taxonomy Code_12',
                                           'Healthcare Provider Taxonomy Code_13',
                                           'Healthcare Provider Taxonomy Code_14',
                                           'Healthcare Provider Taxonomy Code_15'])
    taxonomy_code = taxonomy_code.rename(columns={'value':'taxonomy_code'})
    tax_temp = pd.merge(taxonomy_codes,taxonomy_switches, on=['NPI','code_number'])[['NPI','taxonomy_switch','taxonomy_code']]
    chunk = taxonomy.append(tax_temp)
    chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)

In [ ]:
db.execute('CREATE INDEX npi ON npi(npi)')

In [44]:
db.close()

In [42]:
db = sqlite3.connect('data/hop_team.sqlite')

taxonomy = pd.read_csv('data/nucc_taxonomy_220.csv')

taxonomy.columns = [x.lower().replace(' ', '_') for x in taxomony.columns]

taxonomy.to_sql('taxonomy', db, if_exists = 'append', index = False)


db.execute('CREATE INDEX Code ON taxonomy(Code)')  
db.close()

NameError: name 'taxomony' is not defined